In [1]:
import pandas as pd
import elasticsearch
from elasticsearch_dsl import Search, A, Q
from elasticsearch_dsl.query import MatchAll, Match

/s/ls4/users/grartem/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


multiterm aggregation
#https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-terms-aggregation.html

In [50]:
es = elasticsearch.Elasticsearch(hosts=[{'host': 'localhost', 'port': 9200}])

In [3]:
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": "",
                        "fuzziness": "AUTO",
                        "fields": [
                            "phrase",#,#,
                            "phrase.ext"
                            #"phrase.bigrams"
                            #"phrase.ngram"
                        ]
                    }
                },
            ],
            "should": [
                {
                    "multi_match": {
                        "query": "",
                        "fuzziness": 1,
                        "fields": [
                            "norm_phrase"
                            #"norm_phrase.bigrams"
                            #"norm_phrase.ngram"
                        ]
                    }
                }
            ]
        }
    },
    "size": 25,
    "sort": {
        "_score": {
            "order": "desc"
        }
    },
}

query = {
    "query": {
        "match_all": {}
    }
}
res = es.search(index="rdrs110722", body = query)

/tmp/ipykernel_11825/3176541370.py:46: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="rdrs110722", body = query)


In [ ]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).query("match_all")
s.execute()

In [156]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).aggs.bucket("UniqueDrugnames", agg_type="terms", field="Drugname_kw")

In [164]:
response = s.execute()

In [165]:
listed = []
#for hit in s[0:3800]:
for hit in s.scan():
    listed.append(hit.to_dict())

In [148]:
res = s.aggs.bucket("UniqueDrugnames", agg_type="terms", field="Drugname_kw").bucket("UniqueDrugnames", agg_type="sum", field="Diseasename_kw"


In [170]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).query("match_all")
s.aggs.bucket("drugs", A("cardinality", field="Drugname_kw"))
res = s.execute()

RequestError: RequestError(400, 'search_phase_execution_exception', 'Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field instead. Alternatively, set fielddata=true on [Drugname_kw] in order to load field data by uninverting the inverted index. Note that this can use significant memory.')

In [123]:
s["UniqueDrugnames"].bucket("tags_per_category")

TypeError: '<' not supported between instances of 'str' and 'int'

In [166]:
pd.DataFrame(listed)

,Drugname_kw,Diseasename_kw
0,[Амиксин],"[герпеса, Герпес]"
1,"[Иммунофлазид, Иммунофлазид]","[ГРВИ, грипа]"
2,"[Гавискон, Гавискон, Гавискон]",NaN
3,[Феназепам],NaN
4,[Виферон],[простуд]
...,...,...
3816,"[Кленбутерол, кленбутерол]",NaN
3817,"[Леденцы Вулкан Лакричные, леденцах лакричных,...",NaN
3818,"[Диафлекс РОМФАРМ, Диафлекс, омепрозол, Диафле...",NaN
3819,"[Целебрекс, цереблекс]",[обострении радикулита]


# простой индекс

In [63]:
s = Search(using=es, index="med800k_scheme2").query("match_all").count()
s

490281

In [75]:
s = Search(using=es, index="med800k_scheme2").query("match_all")
s.execute()[7].to_dict().keys()

dict_keys(['Drugnames', 'Diseasenames', 'Indications_text', 'ADRs_text', 'Indications_kw', 'ADRs_kw', 'ADR_reviews_count', 'Negated_ADE_reviews_count', 'Neg_reviews_count', 'Pos_reviews_count', 'Review_count', 'Review_urls'])

In [55]:

#s = Search(using=es, index="rdrs110722_scheme2").query("match_all")
#s = Search(using=es, index="rdrs110722_scheme2").query("match", Indications="кашель, насморк")
s = Search(using=es, index="med800k_scheme2").query("match_all").count()
s = Search(using=es, index="med800k_scheme2").query("match", Indications="кашель, насморк").count()
query = Q("match", Drugnames='мукалтин') | Q("match", Indications_text='кашель')
s = Search(using=es, index="med800k_scheme2").query(query).source(["Drugnames", "Diseasenames", "Indications_kw", "ADRs_kw", "ADR_reviews_count", 
                                                                   "Negated_ADE_reviews_count", "Neg_reviews_count", "Pos_reviews_countg",
                                                                   "Review_count", "Review_urls"]).sort({'Review_count': {"order": "desc"}})

In [58]:
print(query)

Match(Indications_text='кашель')


In [56]:
listed = []
for hit in s[0:1000]:
#for hit in s.scan():
    listed.append(hit.to_dict())

In [57]:
pd.DataFrame(listed)

,ADR_reviews_count,Diseasenames,Indications_kw,Review_urls,Drugnames,Negated_ADE_reviews_count,Neg_reviews_count,Review_count,ADRs_kw
0,0,,[кашель],"[https://otzovik.com/review_1886100.html, http...",мукалтин,16,16,81,NaN
1,0,,[кашель],"[https://otzovik.com/review_3359897.html, http...",амброксол,4,4,50,NaN
2,0,,[кашель],"[https://otzovik.com/review_2276645.html, http...",лазолван,12,15,42,NaN
3,0,,[кашель],"[https://otzovik.com/review_3652255.html, http...",ацц,13,14,41,NaN
4,0,,[кашель],"[https://otzovik.com/review_5333211.html, http...",геделикс,9,9,39,NaN
...,...,...,...,...,...,...,...,...,...
995,0,,"[боль, боль в ротоглотке (орофарингеальная), к...",[https://otzovik.com/review_4864489.html],"амброксол, асс, гексорал табс с, доктор тайсс",1,1,1,NaN
996,0,"бронхит, стоматит","[кашель, повышенная температура тела, ринорея,...",[https://otzovik.com/review_2651614.html],азитро сандоз,0,0,1,NaN
997,0,,[кашель],[https://otzovik.com/review_1815215.html],"лазолван, мукалтин, фенистил, флюдитек",0,1,1,NaN
998,1,,"[кашель, повышенная температура тела, ринорея,...",[https://otzovik.com/review_3651565.html],"амбробене, аскорил, цитовир",1,1,1,"[нездоровый образ жизни, расстройство сна, чув..."


In [19]:
pd.DataFrame(listed)

,Drugnames,Diseasenames,Indications.text,Indications.kw,ADR_reviews_count,Negated_ADE_reviews_count,Neg_reviews_count,Pos_reviews_countg,Review_count,Review_urls,ADRs.text,ADRs.kw
0,мукалтин,,"кашлем, кашля, кашель сухой, кашель кашель, за...",[кашель],0,16,16,75,81,"[https://otzovik.com/review_1886100.html, http...",NaN,NaN
1,мукалтин,,"кашлем, сухомн кашле, кашля, мокром, кашлять, ...","[кашель, продуктивный кашель]",0,3,4,21,23,"[https://otzovik.com/review_4836435.html, http...",NaN,NaN
2,мукалтин,,NaN,NaN,0,1,1,6,10,"[https://otzovik.com/review_1746950.html, http...",NaN,NaN
3,мукалтин,бронхит,"кашлем, кашля, несильном кашле, кашлять, кашля...",[кашель],0,1,1,5,6,"[https://otzovik.com/review_6532245.html, http...",NaN,NaN
4,мукалтин,простуды,"кашлять, закашляем, кашля, подкашливать, кашле...",[кашель],0,1,1,6,6,"[https://otzovik.com/review_3083048.html, http...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
307,"grippostd herbal, гриппостад травяной, лазолва...",,"кашлять, кашлем, кашля, кашель",[кашель],0,0,0,1,1,[https://otzovik.com/review_146980.html],NaN,NaN
308,"лазолван, мукалтин",аллергии,"кашля, мокрота, кашель, влажный кашель","[кашель, продуктивный кашель]",1,0,1,1,1,[https://otzovik.com/review_834338.html],аллергия,[гиперчувствительность]
309,"мукалтин, пертусин, этиловый спирт",аллергии,"кашле, кашля, закашляли, кашель",[кашель],0,0,0,1,1,[https://otzovik.com/review_5939150.html],NaN,NaN
310,"алтей, амбробене, аскорил, ацц, лазолван, мука...",,"прозрачные, выделения из носа, кашель","[глазные выделения, кашель, ринит]",0,0,0,0,1,[https://otzovik.com/review_2012779.html],NaN,NaN


In [193]:
import json
with open("/s/ls4/users/grartem/SAG_MED/med-corpus-analyser/data/raw/ner_800k_jsonl/ner_800k.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        print(data)
        break

{'text_id': 0, 'text': 'Я люблю пользоваться всякими гелями для душа, особенно, которые приятно пахнут. Кремом-гелем пользовалась впервые. Его нам подарили на работе на 8 марта. Что могу сказать-отличный подарок. Раньше продукцией Dove я практически не пользовалась, разве только мылом, крем-гель мне очень понравился. Нежный, пенящийся, приятный легкий аромат. Не сушит кожу после использования, хорошо увлажняет и питает. Очень вкусно пахнет кокосом и жасмином, ненавязчиво. Пользоваться-сплошное наслаждение. Теперь я проверила его действие на себе и смело могу купить в подарок близким людям и друзьям. Могу порекомендовать крем-гель Dove всем любительницам понежеться в ванне, я думаю, что вам он тоже понравится.', 'entities': {'0': {'entity_id': 0, 'text': 'очень понравился кожу', 'tag': ['Disease:DisTypeBNE-Pos'], 'spans': [{'begin': 277, 'end': 293}]}, '1': {'entity_id': 1, 'text': 'хорошо увлажняет и питает', 'tag': ['Disease:DisTypeBNE-Pos'], 'spans': [{'begin': 373, 'end': 398}]}, '2

In [191]:
data

{'text_id': 0,
 'text': 'Я люблю пользоваться всякими гелями для душа, особенно, которые приятно пахнут. Кремом-гелем пользовалась впервые. Его нам подарили на работе на 8 марта. Что могу сказать-отличный подарок. Раньше продукцией Dove я практически не пользовалась, разве только мылом, крем-гель мне очень понравился. Нежный, пенящийся, приятный легкий аромат. Не сушит кожу после использования, хорошо увлажняет и питает. Очень вкусно пахнет кокосом и жасмином, ненавязчиво. Пользоваться-сплошное наслаждение. Теперь я проверила его действие на себе и смело могу купить в подарок близким людям и друзьям. Могу порекомендовать крем-гель Dove всем любительницам понежеться в ванне, я думаю, что вам он тоже понравится.',
 'entities': {'0': {'entity_id': 0,
   'text': 'очень понравился кожу',
   'tag': ['Disease:DisTypeBNE-Pos'],
   'spans': [{'begin': 277, 'end': 293}]},
  '1': {'entity_id': 1,
   'text': 'хорошо увлажняет и питает',
   'tag': ['Disease:DisTypeBNE-Pos'],
   'spans': [{'begin': 

In [76]:
import requests
import json
data = {
    "Indications": "кашель"
}
r = requests.post("http://localhost:8086/scheme2_request", data = json.dumps(data))
r.text

'[{"ADR_reviews_count":0,"Diseasenames":"","Drugnames":"\\u043c\\u0443\\u043a\\u0430\\u043b\\u0442\\u0438\\u043d","Indications":"\\u043a\\u0430\\u0448\\u0435\\u043b\\u044c","Neg_reviews_count":16,"Negated_ADE_reviews_count":16,"Review_count":81,"Review_urls":"https://otzovik.com/review_1886100.html, https://otzovik.com/review_6015011.html, https://otzovik.com/review_5885616.html, https://otzovik.com/review_4254290.html, https://otzovik.com/review_5514232.html, https://otzovik.com/review_5851979.html, https://otzovik.com/review_2308815.html, https://otzovik.com/review_5038396.html, https://otzovik.com/review_1835724.html, https://otzovik.com/review_4201178.html, https://otzovik.com/review_2064981.html, https://otzovik.com/review_1309551.html, https://otzovik.com/review_3190209.html, https://otzovik.com/review_345680.html, https://otzovik.com/review_3527521.html, https://otzovik.com/review_4229375.html, https://otzovik.com/review_4001407.html, https://otzovik.com/review_6840886.html, htt

In [3]:
import pandas as pd
pd.DataFrame(r.json())

ModuleNotFoundError: No module named 'pandas'